In [1]:
from __future__ import print_function, division

In [2]:
from numpy import *

In [3]:
import scipy.spatial

In [4]:
## Generate a random affine matrix
def gen_random_affine():
    return random.random( (3,4) )

## Generate an affine identity matrix
def gen_identity_affine():
    return identity(4)[:3]

In [5]:
## Let's create some poses. The first (0-th) pose will be the identity.
num_handles = 4
num_poses = 3

poses_handles = []

poses_handles.append( [ gen_identity_affine() for i in range(num_handles) ] )
for pose in range( 1, num_poses ):
    poses_handles.append( [ gen_random_affine() for i in range(num_handles) ] )

## The dimensions of poses_handles are: #handles by #poses by affine 3x4 matrix
poses_handles = asarray( poses_handles )
poses_handles = swapaxes( poses_handles, 0, 1 )
print( poses_handles.shape )

(4, 3, 3, 4)


In [6]:
## Let's create some per-vertex skinning weights. Each vertex has a weight for each handle.
num_vertices = 100
## Start with random weights
weights = random.random( ( num_vertices, num_handles ) )
## Make the weights sum to 1. For each successive weight, scale it by whatever is left.
for h in range( 1, num_handles-1 ):
    weights[:,h] *= (1 - weights[:,:h].sum(axis=1))
## Make the last weight be whatever is left.
weights[:,-1] = (1 - weights[:,:-1].sum(axis=1))

## Overwrite the last num_handles vertices with binary vectors elements
ensure_corners = True
if ensure_corners:
    for h in range( num_handles ):
        weights[-(h+1)] = 0.
        weights[-(h+1),h] = 1.
    
    print( weights[-num_handles:] )

print( "Maximum weight for each handle:", weights.max( axis = 0 ) )

## The dimensions of weights are #vertices by #handles.

## The weights all be positive.
assert weights.min() >= 0

## The weights must sum to one. Otherwise, we should normalize them once.
assert abs( weights.sum( axis = 1 ) - 1. < 1e-7 ).all()

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]
Maximum weight for each handle: [ 1.  1.  1.  1.]


In [7]:
## The lbs() function generates the weighted average transformation.

## For all poses at once.
def lbs_all_poses( poses_handles, weights ):
    ## The weights across handles must sum to one.
    assert abs( weights.sum( axis = 1 ) - 1. < 1e-7 ).all()
    
    return ( weights[:,:,newaxis,newaxis,newaxis] * poses_handles[newaxis,...] ).sum( axis = 1 )

## For one set of handles (e.g. one pose) at a time.
def lbs_one_pose( handles, weights ):
    return lbs_all_poses( handles[:,newaxis,...], weights )

In [8]:
## lbs_one_pose() on pose 0 should give us an identity matrix for every vertex.
lbs_one_pose( poses_handles[:,0], weights )

array([[[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       ..., 
       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]]])

In [9]:
poses = lbs_all_poses( poses_handles, weights )

In [10]:
## Doesn't work because it's in a linear subspace
# pose = lbs_one_pose( poses_handles[:,2], weights )
# hull = scipy.spatial.ConvexHull( pose.reshape( num_vertices, -1 ) )

In [11]:
## Doesn't work because it's in a linear subspace
# hull = scipy.spatial.ConvexHull( poses.reshape( num_vertices, -1 ) )

In [12]:
## What are the dimensions of our poses? It should be #verts by #poses by 3x4 affine matrix.
print( poses.shape )
## Flattened it should be #vertices by (#poses*3*4)
X = poses.reshape( num_vertices, num_poses * 3 * 4 )
print( X.shape )

(100, 3, 3, 4)
(100, 36)


In [13]:
## Use SVD to get the lower dimensional space.
## The number of non-zero singular values is the number of handles for perfect reconstruction.
U, s, V = linalg.svd( X-average(X,axis=0)[newaxis,:], full_matrices = False, compute_uv = True )

In [14]:
## What are the singular values?
s.round(4)

array([ 4.0275,  2.5605,  2.3376,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ])

In [15]:
## Find the first index less than a threshold:
len(s)-searchsorted( s[::-1], 1e-6 )

3

In [16]:
## SVD reconstructs the original data
abs( X - (( U @ diag(s) @ V )+average(X,axis=0)[newaxis,:]) ).max()

4.4408920985006262e-16

In [28]:
## Formalize the above with functions.
def uncorrellated_pose_space( poses, threshold = 1e-6 ):
    X = poses.reshape( num_vertices, num_poses * 3 * 4 )
    
    ## Subtract the average.
    Xavg = average( X, axis = 0 )[newaxis,:]
    Xp = X - Xavg

    U, s, V = linalg.svd( Xp, full_matrices = False, compute_uv = True )
    
    ## The first index less than threshold
    stop_s = len(s) - searchsorted( s[::-1], threshold )
    
    def restore( uncorrellated_poses ):
        return ( ( uncorrellated_poses @ V[:stop_s] ) + Xavg ).reshape( -1, num_poses, 3, 4 )
    
    return Xp @ V[:stop_s].T, restore

In [29]:
## Vertify that we can go back and forth with these functions
uncorrelated_poses, restore = uncorrellated_pose_space( poses )
print( uncorrelated_poses.shape )
abs( restore( uncorrelated_poses ) - poses ).max()

(100, 3)
(1, 36)


5.5511151231257827e-16

In [19]:
uncorrelated_poses

array([[  6.11219111e-01,  -4.36293510e-02,   8.42526228e-02],
       [  1.61388431e-01,  -2.40742765e-01,   1.36475133e-01],
       [ -6.67196562e-02,  -2.41144317e-01,   8.28379613e-02],
       [ -3.53409674e-01,  -1.03541931e-01,  -3.71754116e-02],
       [ -5.37167419e-01,  -8.75356693e-02,  -3.67364266e-02],
       [  6.11985323e-01,  -6.94840607e-02,   9.42488749e-02],
       [ -7.34939544e-01,  -3.63687571e-01,  -3.47740015e-02],
       [ -4.38262535e-02,   2.12506275e-01,  -4.11615964e-01],
       [  6.33517048e-01,  -6.36336819e-02,   1.00470103e-01],
       [  6.14389618e-01,  -6.39249099e-03,   1.08849158e-01],
       [ -2.86179058e-01,  -9.20167775e-02,  -1.69643785e-01],
       [ -3.58606166e-01,  -2.02636163e-01,  -2.31335300e-01],
       [  4.44295488e-01,  -3.51219025e-02,  -1.50284633e-01],
       [ -3.97631297e-01,   3.29854268e-01,   1.80538198e-01],
       [  3.01193501e-02,  -1.63718286e-01,   1.00857084e-01],
       [ -5.91976884e-02,   1.12869004e-01,   1.7339183

In [20]:
## Now let's use the convex hull on the uncorrelated poses.
hull = scipy.spatial.ConvexHull( uncorrelated_poses )
print( len( hull.vertices ) )
print( hull.vertices )

4
[96 97 98 99]


In [21]:
poses_handles_restored = restore( uncorrelated_poses[ hull.vertices ] )
print( poses_handles_restored.shape )
poses_handles_restored.round(4)

(4, 3, 3, 4)


array([[[[ 1.    ,  0.    , -0.    ,  0.    ],
         [ 0.    ,  1.    ,  0.    ,  0.    ],
         [ 0.    ,  0.    ,  1.    ,  0.    ]],

        [[ 0.5383,  0.4509,  0.5815,  0.7645],
         [ 0.7244,  0.3916,  0.294 ,  0.0329],
         [ 0.1152,  0.8129,  0.9089,  0.2088]],

        [[ 0.098 ,  0.5627,  0.9485,  0.9391],
         [ 0.9325,  0.5595,  0.7009,  0.0275],
         [ 0.2199,  0.0972,  0.4408,  0.1332]]],


       [[[ 1.    ,  0.    ,  0.    , -0.    ],
         [ 0.    ,  1.    ,  0.    ,  0.    ],
         [ 0.    ,  0.    ,  1.    ,  0.    ]],

        [[ 0.5782,  0.5148,  0.581 ,  0.8704],
         [ 0.0812,  0.2603,  0.454 ,  0.2381],
         [ 0.6445,  0.1032,  0.3344,  0.1156]],

        [[ 0.9509,  0.6309,  0.5878,  0.7895],
         [ 0.3091,  0.856 ,  0.1497,  0.8952],
         [ 0.2308,  0.9543,  0.0233,  0.8069]]],


       [[[ 1.    , -0.    ,  0.    , -0.    ],
         [ 0.    ,  1.    ,  0.    ,  0.    ],
         [ 0.    ,  0.    ,  1.    ,  0.    

In [22]:
print( poses_handles.shape )
poses_handles

(4, 3, 3, 4)


array([[[[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  1.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  1.        ,  0.        ]],

        [[ 0.63319812,  0.05190949,  0.64953769,  0.891664  ],
         [ 0.07747892,  0.5310581 ,  0.02506164,  0.426007  ],
         [ 0.06830536,  0.13588485,  0.77438178,  0.41180536]],

        [[ 0.90070341,  0.27218822,  0.15915252,  0.76117287],
         [ 0.4850491 ,  0.38408144,  0.4277015 ,  0.503789  ],
         [ 0.3807526 ,  0.8447042 ,  0.88070627,  0.76023809]]],


       [[[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  1.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  1.        ,  0.        ]],

        [[ 0.34450283,  0.62586702,  0.7721408 ,  0.86832167],
         [ 0.79734644,  0.20087059,  0.16028609,  0.76860263],
         [ 0.17334852,  0.48301027,  0.79795303,  0.51069208]],

        [[ 0.26048207,  0.106867  ,  0.7716

In [23]:
## Find the closest handle in the original data, pose_handles, to the restored data.
for i, handles in enumerate( poses_handles ):
    diffs = ( abs( poses_handles_restored - handles[newaxis,...] ).reshape( poses_handles_restored.shape[0], -1 ).sum( axis = 1 ) )
    print( "Original data index", i, "best matches restored data index", diffs.argmin(), "with error", diffs.min() )

Original data index 0 best matches restored data index 3 with error 1.96398654668e-15
Original data index 1 best matches restored data index 2 with error 3.12997073922e-15
Original data index 2 best matches restored data index 1 with error 2.56999469357e-15
Original data index 3 best matches restored data index 0 with error 3.07898328166e-15


In [24]:
## Find the closest handle in the restored data to the original pose_handles.
for i, handles in enumerate( poses_handles_restored ):
    diffs = ( abs( poses_handles - handles[newaxis,...] ).reshape( poses_handles.shape[0], -1 ).sum( axis = 1 ) )
    print( "Restored data index", i, "best matches original data index", diffs.argmin(), "with error", diffs.min() )

Restored data index 0 best matches original data index 3 with error 3.07898328166e-15
Restored data index 1 best matches original data index 2 with error 2.56999469357e-15
Restored data index 2 best matches original data index 1 with error 3.12997073922e-15
Restored data index 3 best matches original data index 0 with error 1.96398654668e-15


In [25]:
## Plotting in 3D. This only works because PCA found our data was in 3D.
%matplotlib tk
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*uncorrelated_poses.T)
plt.show()